In [1]:
import math

import numpy as np

from matplotlib import pyplot as plt
from IPython.display import HTML

from matk.utils.animation import animate_frames

%matplotlib inline

In /home/vsydorskyi/anaconda3/envs/venv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/vsydorskyi/anaconda3/envs/venv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/vsydorskyi/anaconda3/envs/venv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/vsydorskyi/anaconda3/envs/venv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /

In [2]:
import math

from typing import Tuple

import numpy as np

from matk.models.base_model import _BaseModel
from matk.utils.geometry import angle_between_points


def coord_to_from_point(position, distination, step_size, clipping, to=True):
    angle = angle_between_points(position, distination) 
    abs_angle = np.clip(abs(angle), 0, clipping)
    if angle < 0:
        angle = -abs_angle
    else:
        angle = angle
    rad = math.radians(angle)
    dx = math.cos(rad) * step_size
    dy = math.sin(rad) * step_size
    if not to:
        dx = -dx
        dy = -dy
    else:
        if np.abs(position - distination).max() < 0.5:
            return 0, 0
        
    return dx, dy

class FlockingModel(_BaseModel):
    def __init__(
        self,
        n_points: int,
        field_size: Tuple[int, int],
        step_size: int,
        view: float = 5,
        minimum_separation: float = 20,
        max_align_turn: float = 30,
        max_cohere_turn: float = 20,
        max_separate_turn: float = 10
    ):
        super().__init__(
            n_points=n_points,
            field_size=field_size,
            step_size=step_size,
            keep_trajoctories=False,
        )
        self.view = view
        self.minimum_separation = minimum_separation
        self.max_align_turn = max_align_turn
        self.max_cohere_turn = max_cohere_turn
        self.max_separate_turn = max_separate_turn
        
        self.points = []
        self.is_placed = []

    def create_field(self):
        point_coords = [
            np.random.randint(0, f_size, self.n_points)
            for f_size in self.field_size
        ]
        point_coords = np.stack(point_coords, axis=-1).astype(float)

        self.points.append(point_coords)
        self.markup_field(point_coords)
        
    def compute_step_to_flockmate_heading(self, current_coord, coords):
        distances = np.linalg.norm( coords - current_coord[None, :], axis=-1)
        flock = coords[distances < self.view]
        if len(flock) < 2:
            return current_coord
        head = flock.mean(0)
        dx, dy = coord_to_from_point(
            current_coord, 
            head, 
            self.step_size,
            self.max_align_turn
        )
        current_coord[0] += dx
        current_coord[1] += dy
        return current_coord
    
    def compute_step_out_of_neighbour(self, current_coord, neighbour_coord):
        dx, dy = coord_to_from_point(
            current_coord, 
            neighbour_coord, 
            self.step_size, 
            self.max_separate_turn,
            to=False
        )
        current_coord[0] += dx
        current_coord[1] += dy
            
        return current_coord
    
    def compute_step_to_neighbour(self, current_coord, neighbour_coord):
        dx, dy = coord_to_from_point(
            current_coord, 
            neighbour_coord, 
            self.step_size, 
            self.max_cohere_turn
        )
        current_coord[0] += dx
        current_coord[1] += dy
            
        return current_coord
    
    def compute_dist_to_nearby(self, current_coord, current_coord_id, coords):
        all_p_exclude_current = np.delete(coords, current_coord_id, 0) 
        distances = np.linalg.norm( all_p_exclude_current - current_coord[None, :], axis=-1)
        argmin_dist = np.argmin(distances)
        nearest_point = all_p_exclude_current[argmin_dist]
        smallest_dist = distances[argmin_dist]
        return smallest_dist, nearest_point

    def step(self):
        current_coords = self.points[-1].copy()
        new_coords = np.zeros_like(current_coords)

        for i in range(current_coords.shape[0]):
            current_point = current_coords[i]
            smallest_dist, nearest_point = self.compute_dist_to_nearby(
                current_point,
                i,
                current_coords
            )
            if smallest_dist < self.minimum_separation:
                new_point = self.compute_step_out_of_neighbour(current_point, nearest_point)
                new_point = self.continious_boarder_mode(new_point)
            else:
                new_point = self.compute_step_to_flockmate_heading(current_point, current_coords)
                new_point = self.continious_boarder_mode(new_point)
                _, nearest_point = self.compute_dist_to_nearby(
                    new_point,
                    i,
                    current_coords
                )
                new_point = self.compute_step_to_neighbour(new_point, nearest_point)
                new_point = self.continious_boarder_mode(new_point)
                
            new_coords[i] = new_point
           
        self.points.append(new_coords)
        self.markup_field(new_coords)

    def reset_partial(self):
        self.points = []
        self.stop = False


In [3]:
model = FlockingModel(
    100, 
    (64, 64), 
    1,
    view=64,
    minimum_separation=2,
    max_align_turn=100,
    max_cohere_turn=100,
    max_separate_turn=4.5
)

In [4]:
model.run_n_steps(1000)

100%|██████████| 1000/1000 [00:06<00:00, 154.96it/s]


In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(model[-1])
plt.show()

In [ ]:
ani = animate_frames(model[-200:], figsize=(6,6))
HTML(ani.to_jshtml())